In [0]:
import requests
import json
import pyspark.sql.functions as F
from pyspark.sql.types import *
import pyspark
from pyspark.sql.functions import sum,avg,max,min,mean,count

In [0]:
spark.conf.set(
  "")
spark.conf.set(
  "")
df = spark.read.csv("wasbs://test@sentimentdatacs4225.blob.core.windows.net/feb-1.csv")

In [0]:
df = spark.read.csv("wasbs://test@sentimentdatacs4225.blob.core.windows.net/feb-1.csv")
df = df.sample(False, 0.16, 4225)
df.count()
# df = df.limit(10)
# df.count()
# df.show()

Out[36]: 10000

In [0]:
  def get_sentiment(sentence):
    url = 'https://cs4225sentimenttest-1.azurewebsites.net/sentiment'
    myobj = {'data': sentence}
    try:
      x =  requests.post(url, data = myobj).json()
    except:
      return "-1.00"
#     print(x)
    return x[0]["confidenceScores"]["positive"],  x[0]["confidenceScores"]["neutral"],x[0]["confidenceScores"]["negative"]
  

In [0]:
 def postiveFunction(value):

  if(value == None or value == ""):
    return "-1.00"
  res =  get_sentiment(value)
  return str(res).replace("(","").replace(")","")
udfsomefunc = F.udf(postiveFunction, StringType())
# udfsomefunc = F.udf(postiveFunction, StringType())


In [0]:
#  def test(line):
#   res =  postiveFunction(line._c3)
# #   print(line._c0)
#   return (line._c0,line._c1,line._c2,line._c3,str(res).replace("(","").replace(")",""))
udfsomefunc = F.udf(postiveFunction, StringType())
df3 = df.withColumn("sentiment",udfsomefunc(df._c3))
df3.write.format("csv").save("wasbs://test@sentimentdatacs4225.blob.core.windows.net/feb-sentiment-" + "1" + ".csv")

In [0]:
df3 = spark.read.csv("wasbs://test@sentimentdatacs4225.blob.core.windows.net/feb-sentiment-1.csv")
df3.show()

+----------+---+--------------+--------------------+----------------+
 _c0|_c1| _c2| _c3| _c4|
+----------+---+--------------+--------------------+----------------+
2020-02-01| en| India|student return ch...|0.93, 0.06, 0.01|
2020-02-01| en| India|indian home 👍👏...|0.05, 0.94, 0.01|
2020-02-01| en| India|coronavirusindian...|0.01, 0.45, 0.54|
2020-02-01| en|United Kingdom|coronavirus feb -...| 0, 1, 0|
2020-02-01| en| Hong Kong|medical workers h...| 0, 0.2, 0.8|
2020-02-01| en| France|latest free world...| 0.92, 0.08, 0|
2020-02-01| en| Germany|china reports out...| 0, 0.01, 0.99|
2020-02-01| en| Singapore|father can’t touc...| 0, 0.01, 0.99|
2020-02-01| en| Philippines|they’re calling h...|0.16, 0.79, 0.05|
2020-02-01| en| Malaysia|dear created dedi...|0.14, 0.84, 0.02|
2020-02-01| en| Thailand|thai embassy beij...| 0, 0.86, 0.14|
2020-02-01| en|United Kingdom|breaking confirme...| 0, 0.97, 0.03|
2020-02-01| en| Hong Kong|actually predicta...|0.01, 0.18, 0.81|
2020-02-01| en| India|disgusting behavi...|0.01, 0.02, 0.97|
2020-02-01| en| India|really sad pakist...| 0, 0.02, 0.98|
2020-02-01| en|United Kingdom|new modelling stu...| 0.01, 0.6, 0.39|
2020-02-01| en| Canada|i’m tired hearing...| 0, 0, 1|
2020-02-01| en| India|update air india ...|0.01, 0.96, 0.03|
2020-02-01| en| Hong Kong|thousands medical...| 0, 0.16, 0.84|
2020-02-01| en| Hong Kong|“if stay home don...| 0, 0.95, 0.05|
+----------+---+--------------+--------------------+----------------+
only showing top 20 rows

In [0]:
df4 = df3.filter(df3._c4 != '-1.00')
split_col = pyspark.sql.functions.split(df4['_c4'], ',')
df4 = df4.withColumn('postive', split_col.getItem(0)).withColumn('neutral', split_col.getItem(1)).withColumn('negative', split_col.getItem(2))
# df4 = df3.withColumn('NAME2', split_col.getItem(1))

In [0]:
df5 = df4.groupby("_c0","_c2").agg(avg("postive").alias("postive_avg")
                                   ,avg("neutral").alias("neutral_avg"),
                                   avg("negative").alias("negative_avg"),
                                   count("postive").alias("TweetCount"))

In [0]:
df5.show()

+----------+--------------+-------------------+-------------------+-------------------+----------+
 _c0| _c2| postive_avg| neutral_avg| negative_avg|TweetCount|
+----------+--------------+-------------------+-------------------+-------------------+----------+
2020-02-01| Thailand|0.14539473684210522|0.42640350877192984| 0.4282017543859648| 228|
2020-02-01| Philippines|0.14225433526011555| 0.3696531791907514| 0.4880924855491329| 519|
2020-02-01| Germany|0.10753164556962026| 0.3632911392405063| 0.5291772151898734| 158|
2020-02-01| China|0.14679487179487177|0.41551282051282046|0.43769230769230766| 234|
2020-02-01| Hong Kong|0.06754934210526313|0.33519736842105263| 0.5972532894736842| 1216|
2020-02-01| Malaysia|0.17273732718894014|0.27036866359447015| 0.5568940092165899| 1085|
2020-02-01|United Kingdom|0.11144254278728602|0.42572127139364313|0.46283618581907093| 409|
2020-02-01| India|0.11231560891938243| 0.4307632933104631| 0.456921097770154| 3498|
2020-02-01| Canada|0.11424552429667514| 0.3999104859335038| 0.4858439897698211| 782|
2020-02-01| France|0.14123076923076922| 0.3670769230769231|0.49169230769230776| 260|
2020-02-01| United States|0.10110038610038605|0.40367760617760623| 0.4952220077220079| 1036|
2020-02-01| Singapore| 0.1139171974522293|0.36070063694267507| 0.5253821656050955| 314|
2020-02-01| Japan|0.14707920792079204| 0.3999009900990098| 0.453019801980198| 202|
2020-02-01| Sweden|0.10056603773584907| 0.3852830188679245| 0.5141509433962265| 53|
+----------+--------------+-------------------+-------------------+-------------------+----------+

In [0]:
server_name = "jdbc:sqlserver://cs4225database.database.windows.net"
database_name = "sentimentResult"
url = server_name + ";" + "databaseName=" + database_name + ";"

table_name = "sentimentResult"
username = ""
password = "" # Please specify password here

In [0]:
df5.write.format("jdbc") \
  .mode("append") \
  .option("url", "") \
  .option("dbtable", "") \
  .option("user", "") \
  .option("password", "") \
  .save()

In [0]:
dbutils.fs.mount(
  source = "wasbs://test@sentimentdatacs4225.blob.core.windows.net",
  mount_point = "/mnt/test",
  extra_configs = {"D"})

Out[6]: True

In [0]:
df = spark.read.csv("/mnt/test/feb-1.csv")
df = df.sample(False, 0.1, 7777)
df.count()

Out[7]: 10688

In [0]:
udfsomefunc = F.udf(postiveFunction, StringType())
df3 = df.withColumn("sentiment",udfsomefunc(df._c3))
df3.cache()
df3.show()
df3.count()

+----------+---+--------------+--------------------+----------------+
 _c0|_c1| _c2| _c3| sentiment|
+----------+---+--------------+--------------------+----------------+
2020-02-01| en|United Kingdom|coaches collectin...| 0, 0.02, 0.98|
2020-02-01| en| Niger|countries suspend...|0.38, 0.28, 0.34|
2020-02-01| en| Indonesia|father can’t touc...| 0, 0.01, 0.99|
2020-02-01| en| India|christian girl re...|0.07, 0.18, 0.75|
2020-02-01| en| Hong Kong|recent opinion po...|0.65, 0.28, 0.07|
2020-02-01| en| France|australia shut do...| 0, 0.07, 0.93|
2020-02-01| en| United States|'highly pathogeni...| 0, 0.06, 0.94|
2020-02-01| en| India|bse banknifty nif...| 0, 0.09, 0.91|
2020-02-01| en| Indonesia|father can’t touc...| 0, 0.01, 0.99|
2020-02-01| en| Philippines|wuhan nurse revea...| 0.8, 0.14, 0.06|
2020-02-01| en| India|stupidity display...| 0, 0.04, 0.96|
2020-02-01| en| Australia|deadly bird flu o...|0.01, 0.05, 0.94|
2020-02-01| en| Niger| sad shit 😷🤒| 0, 0, 1|
2020-02-01| en| Canada|solskjær’s incred...|0.04, 0.93, 0.03|
2020-02-01| en| Pakistan|breaking coronavi...| 0, 0.01, 0.99|
2020-02-01| en| Thailand|members recently ...| 0, 0.43, 0.57|
2020-02-01| en| United States|wuhan coronavirus...| 0, 0.02, 0.98|
2020-02-01| en| United States|major american ai...| 0, 0.03, 0.97|
2020-02-01| en| Hong Kong|ccp propagandizes...| 0, 0, 1|
2020-02-01| en| India|think subjct coro...| 0, 0.1, 0.9|
+----------+---+--------------+--------------------+----------------+
only showing top 20 rows

In [0]:
df3.count()

In [0]:
dbutils.fs.unmount("/mnt/test")

/mnt/test has been unmounted.
Out[46]: True

In [0]:
for i in range(1, 2):
  df = spark.read.csv("wasbs://feb@sentimentdatacs4225.blob.core.windows.net/feb-" + str(i) + ".csv")
  total = df.count()
  ratio = 10000/total;
  df = df.sample(False, ratio, 4225)
  print("Current Day" + str(i))
  print("the count for df")
  df.count()
  udfsomefunc = F.udf(postiveFunction, StringType())
  df3 = df.withColumn("sentiment",udfsomefunc(df._c3))
  intermeiatefilename = "wasbs://feb@sentimentdatacs4225.blob.core.windows.net/feb-sentiment-" + str(i) + ".csv"
  df3.write.format("csv").save(intermeiatefilename)
  df3 = spark.read.csv(intermeiatefilename)
  df4 = df3.filter(df3._c4 != '-1.00')
  split_col = pyspark.sql.functions.split(df4['_c4'], ',')
  df4 = df4.withColumn('postive', split_col.getItem(0)).withColumn('neutral', split_col.getItem(1)).withColumn('negative', split_col.getItem(2))
  df5=df4.groupby("_c0","_c2").agg(avg("postive").alias("postive_avg"),avg("neutral").alias("neutral_avg"),avg("negative").alias("negative_avg"),count("postive").alias("TweetCount"))
  df5.write.format("jdbc") \
  .mode("append") \
  .option("url", "") \
  .option("dbtable", "") \
  .option("user", "") \
  .option("password", "") \
  .save()

Current Day1
the count for df